<a href="https://colab.research.google.com/github/sadnow/Simplified-CLIP-Guided-Diffusion-Colab/blob/main/CLIP_Guided_Diffusion_HQ_256x256.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simplified CLIP Guided Diffusion with (Planned) Real-ESRGAN upscaling
**Alpha 0.5 Release**
*Real-ESRGAN support is still in planned development.*
---

This fork is by sadnow. If you are loading the Notebook directly from this url (https://colab.research.google.com/github/sadnow/Simplified-CLIP-Guided-Diffusion-Colab/blob/main/CLIP_Guided_Diffusion_HQ_256x256.ipynb), you are already receiving the latest updates (kek). In case I mess up a link, the GitHub can also be accessed at Updates can be found on: (https://github.com/sadnow/Simplified-CLIP-Guided-Diffusion-Colab). I welcome any feedback.

---

Modifications so far: 

-Colab forms implementation

-Load dependencies from drive

-Added 512 model toggle (you need a lot of vram)

Planned features: Real-ESRGAN upscaling, 3d photo inpainting, more settings

---
This notebook was brought to you by AnimationKit AI. It upscales videos using Real-ESRGAN and offers RIFE motion smoothing / interpolation. Very early in development, has some big updates coming.

---
## Credits:

"[Original Diffusion notebook] [b]y Katherine Crowson (https://github.com/crowsonkb, https://twitter.com/RiversHaveWings). It uses OpenAI's 256x256 unconditional ImageNet diffusion model (https://github.com/openai/guided-diffusion) together with CLIP (https://github.com/openai/CLIP) to connect text prompts with images."

This notebook was forked from the following Notebook: https://twitter.com/rivershavewings/status/1419674445044539400?lang=en

In [ ]:
# @title Licensed under the MIT License

# Copyright (c) 2021 Katherine Crowson

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

In [ ]:
#@markdown #Check GPU
#@markdown Factory reset if you get below a P100 and want to do a 512 image_size

# Check the GPU status

!nvidia-smi

In [ ]:
def dir_make(a):
  import os.path
  from os import path
  if not path.exists(a):
    print("Creating"+a+"...")
    !mkdir -p $a

def dir_check(a):
  import os.path
  from os import path
  if not path.exists(a):
    return False
  else:
    return True

def file_check(a):
  import os.path
  from os import path
  if not path.isfile(a):
    return False
  else:
    return True

class anKit:
  #def __init__(self):
  #  pass
  import os.path
  from os import path
  #video tools
  def video_sortFrames(sourceframes, destframes):  #takes frames from input folder, moves to init_frame_storage
    dir_make(sourceframes)
    dir_check(destframes)
    %cd $sourceframes
    print("Copying frames to "+destframes+" for processing...")
    !find -maxdepth 1 -name '*.png' -print0 | xargs -0 cp -t $destframes
    %cd $destframes
    !find . -type f -name "*.png" -execdir bash -c 'mv "$0" "${0##*_}"' {} \;  #removes anything not numbers
    #print("Padding filenames in "+destframes+".")
    !rename 's/\d+/sprintf("%05d",$&)/e' *  #adds padding to numbers
    print("Finished copying frames to "+destframes+".")
  def video_splitFrames(sourcefile, destframes):
    check_file(sourcefile)
    dir_check(destframes)
    !ffmpeg -y -r 1 -i $sourcefile -r 1 $destframes/frame%05d.png
  def frames2video(sourceframes,fps,outputmp4):
    dir_make(sourceframes)
    %cd $sourceframes
    #!ffmpeg -r $fps -i '%d_out.png' $outputmp4
    !ffmpeg -y -f image2 -pattern_type glob -i '*.png' $outputmp4
  def video_runUpscale(esrgan,mpath,scale,input,output):
    dir_make(input)
    dir_check(output)
    %cd $esrgan
    #print(mpath)
    !python inference_realesrgan.py --model_path $mpath --netscale $scale --input $input --output $output
    %cd output
    !find . -type f -name "*_*.png" -execdir bash -c 'mv "$0" "${0##*_}"' {} \;  #removes anything not numbers from filename - needed for rife-frame
  def video_compress(input,effects,quality,output):  #needs portable
    !ffmpeg -y -i $input $visual_effects -c:v hevc_nvenc -rc vbr -cq $constant_quality -qmin $constant_quality -qmax $constant_quality -b:v 0 $output_path_mp4
  def RIFE_video(fps,exp,input,scale,output):
    %cd /content/Practical-RIFE
    !python3 /content/Practical-RIFE/inference_video.py --fps=$fps --exp=$exp --video=$input --scale $scale --output=$output
  def RIFE_frames(fps,exp,input,scale,output):  #not currently working right
    %cd /content/Practical-RIFE
    input = input + '/'
    !python3 /content/Practical-RIFE/inference_video.py --fps=$fps --exp=$exp --img=$input --scale $scale --output=$output
  def detect_fps(input): #needs portable
    import re
    fps_ffprobe = !ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=avg_frame_rate $input
    fps_unfinished = [str(i) for i in fps_ffprobe] # Converting integers into strings
    fps_unfinishedTwo = str("".join(fps_unfinished)) # Join the string values into one string
    numbers = re.findall('[0-9]+', fps_unfinishedTwo)
    newNum = numbers[0:1]
    strings = [str(integer) for integer in newNum]
    a_string = "".join(strings)
    fps = int(a_string)
    #print("Detected FPS is",fps)
    return fps
  def detect_duration(input):  #needs portable
    import re
    duration_ffprobe = !ffprobe -v error -select_streams v:0 -show_entries stream=duration -of default=noprint_wrappers=1:nokey=1 $input
    duration_unfinished = [str(i) for i in duration_ffprobe] # Converting integers into strings
    duration_unfinishedTwo = str("".join(duration_unfinished)) # Join the string values into one string
    numbers = re.findall('[0-9]+', duration_unfinishedTwo)
    newNum = numbers[0:1]
    strings = [str(integer) for integer in newNum]
    a_string = "".join(strings)
    duration = int(a_string)
    #print("Detected duration INTEGER (in seconds) is",duration)
    return duration
  def exp_calc(): #needs portable
    import numpy as np
    a = measured_fps * measured_duration
    b = target_fps * target_length_seconds
    c = b / a
    l = np.log(c) / np.log(2)
    print("Un-rounded --exp is",l)
    x = round(l)
    if x < 1:
      x = 1
    print("Rounding up to an --exp of ",x)
    return x
  def image_upscale(model_path,scale,input,output):
    %cd /content/Real-ESRGAN/
    #print(mpath)
    !python inference_realesrgan.py --model_path $model_path --netscale $scale --input $input --output $output
    %cd /content/
  #
  #
  #
  #installation for Colab (untested on other platforms)
  #
  #
  def install_ESRGAN():
    %cd /content/
    print("Installing libraries for Real-ESRGAN upscaling.")
    !git clone https://github.com/xinntao/Real-ESRGAN.git
    %cd /content/Real-ESRGAN
    !pip install basicsr -q
    !pip install facexlib -q
    !pip install gfpgan -q
    !pip install -r requirements.txt -q
    %cd /content/Real-ESRGAN
    !python setup.py develop -q
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth -P experiments/pretrained_models
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth -P experiments/pretrained_models
    print("Finished Installing libraries for Real-ESRGAN upscaling.")
    %cd /content/
    #
  def install_RIFE():
    %cd /content/
    print("Installing libraries for RIFE motion smoothing.")
    !git clone https://github.com/hzwer/Practical-RIFE Practical-RIFE
    !gdown --id 1O5KfS3KzZCY3imeCr2LCsntLhutKuAqj
    #%cd /content/Practical-RIFE
    !7z e RIFE_trained_model_v3.8.zip
    #!7z e /content/Practical-RIFE/RIFE_trained_model_v3.8.zip
    #!7z e /content/RIFE_trained_model_v3.8.zip
    !mkdir /content/Practical-RIFE/train_log
    !mv *.py /content/Practical-RIFE/train_log/
    !mv *.pkl /content/Practical-RIFE/train_log/
    %cd /content/Practical-RIFE/
    !gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc #useless - mp4 demo
    !pip3 install -r requirements.txt
    print("Finsihed Installing libraries for RIFE motion smoothing.")
    #  
    %cd /content/
  def depcrecated_installOldRIFE():
    print("Installing libraries for RIFE motion smoothing.")
    !git clone https://github.com/hzwer/arXiv2020-RIFE RIFE
    !gdown --id 1wsQIhHZ3Eg4_AfCXItFKqqyDMB4NS0Yd
    !7z e RIFE_trained_model_HDv2.zip
    !mkdir /content/RIFE/train_log
    !mv *.pkl /content/RIFE/train_log/
    %cd /content/RIFE/
    !gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc
    !pip3 install -r requirements.txt
    print("Done.")
    print("Finsihed Installing libraries for RIFE motion smoothing.")
  %cd /content/
    
###

#@markdown Install, import, load more functions (Required)
##mount_google_drive = True #@param {type:"boolean"}
load_from_drive = True #@param {type:"boolean"}
drive_install_location = "/content/drive/MyDrive/z_Colab/Simplified-Diffusion_CLIP"#@param {type:"string"}

#@markdown `load_from_drive` is optional. It will save the big files to your specified `drive_install_location`. Next time you run this cell, it will load those files from your Drive. Can save you a minute or two. 

load_512_model = True #@param {type:"boolean"}
load_esrgan = True #@param {type:"boolean"}

def dl_dependencies():
  print("Downloading dependencies...")
  !git clone https://github.com/openai/CLIP
  !git clone https://github.com/crowsonkb/guided-diffusion
  # Download the diffusion model
  !curl -OL 'https://openaipublic.blob.core.windows.net/diffusion/jul-2021/256x256_diffusion_uncond.pt'
  if load_512_model:
    !curl -OL --http1.1 'https://the-eye.eu/public/AI/models/512x512_diffusion_unconditional_ImageNet/512x512_diffusion_uncond_finetune_008100.pt'


def install_dependencies():
    !pip install -e ./CLIP -q
    !pip install -e ./guided-diffusion -q
    !pip install lpips -q
    if load_esrgan:
      anKit.install_ESRGAN()

# Install dependencies loading from Drive if checked
clip_exists = dir_check('/content/CLIP')
if not clip_exists:
  print("Installing dependencies...")
  if load_from_drive:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_install_location_clip = drive_install_location + "/CLIP"
    drive_install_location_exists = dir_check(drive_install_location_clip)
    if not drive_install_location_exists:  #if driveinstalllocation isnt present
      dir_make(drive_install_location)
      print("Downloading dependencies...")
      dl_dependencies()
      print("Copying files to drive_install_location...")
      !rsync -av --progress /content/ $drive_install_location --exclude drive
    else:
      #this means the drive dir exists
      print("Loading dependencies from drive (Drive to /content/)...")
      !rsync -av --progress $drive_install_location/ /content/ --exclude drive
  else:
    dl_dependencies()
else:
  print("/content/CLIP is already present. Skipping dependency download...")
install_dependencies()



# Imports

import gc
import io
import math
import sys

from IPython import display
import lpips
from PIL import Image
import requests
import torch
from torch import nn
from torch.nn import functional as F
from torchvision import transforms
from torchvision.transforms import functional as TF
from tqdm.notebook import tqdm

sys.path.append('./CLIP')
sys.path.append('./guided-diffusion')

import clip
from guided_diffusion.script_util import create_model_and_diffusion, model_and_diffusion_defaults


# Define necessary functions

def fetch(url_or_path):
    if str(url_or_path).startswith('http://') or str(url_or_path).startswith('https://'):
        r = requests.get(url_or_path)
        r.raise_for_status()
        fd = io.BytesIO()
        fd.write(r.content)
        fd.seek(0)
        return fd
    return open(url_or_path, 'rb')


def parse_prompt(prompt):
    if prompt.startswith('http://') or prompt.startswith('https://'):
        vals = prompt.rsplit(':', 2)
        vals = [vals[0] + ':' + vals[1], *vals[2:]]
    else:
        vals = prompt.rsplit(':', 1)
    vals = vals + ['', '1'][len(vals):]
    return vals[0], float(vals[1])


class MakeCutouts(nn.Module):
    def __init__(self, cut_size, cutn, cut_pow=1.):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.cut_pow = cut_pow

    def forward(self, input):
        sideY, sideX = input.shape[2:4]
        max_size = min(sideX, sideY)
        min_size = min(sideX, sideY, self.cut_size)
        cutouts = []
        for _ in range(self.cutn):
            size = int(torch.rand([])**self.cut_pow * (max_size - min_size) + min_size)
            offsetx = torch.randint(0, sideX - size + 1, ())
            offsety = torch.randint(0, sideY - size + 1, ())
            cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]
            cutouts.append(F.adaptive_avg_pool2d(cutout, self.cut_size))
        return torch.cat(cutouts)


def spherical_dist_loss(x, y):
    x = F.normalize(x, dim=-1)
    y = F.normalize(y, dim=-1)
    return (x - y).norm(dim=-1).div(2).arcsin().pow(2).mul(2)


def tv_loss(input):
    """L2 total variation loss, as in Mahendran et al."""
    input = F.pad(input, (0, 1, 0, 1), 'replicate')
    x_diff = input[..., :-1, 1:] - input[..., :-1, :-1]
    y_diff = input[..., 1:, :-1] - input[..., :-1, :-1]
    return (x_diff**2 + y_diff**2).mean([1, 2, 3])


def range_loss(input):
    return (input - input.clamp(-1, 1)).pow(2).mean([1, 2, 3])

In [ ]:


#@markdown Model settings
steps_per_prompt='25' #@param ['25','50','150','250','500','1000','ddim25','ddim50','ddim150','ddim250','ddim500','ddim1000']
#timestep_respacing = steps_per_prompt #same thing, different name
#@markdown Modify this value to change iterations/prompt. `Sacrifices accuracy/alignment for improved quicker runtime.`
diffusion_steps = 1000
image_size = '512' #@param [64,128,256,512]



image_size=int(image_size)
model_config = model_and_diffusion_defaults()
model_config.update({
    'attention_resolutions': '32, 16, 8',
    'class_cond': False,
    'diffusion_steps': diffusion_steps,
    'rescale_timesteps': True,
    'timestep_respacing': steps_per_prompt,
    'image_size': image_size,
    'learn_sigma': True,
    'noise_schedule': 'linear',
    'num_channels': 256,
    'num_head_channels': 64,
    'num_res_blocks': 2,
    'resblock_updown': True,
    'use_fp16': True,
    'use_scale_shift_norm': True,
})


# Load models

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

model, diffusion = create_model_and_diffusion(**model_config)

#512 model addition
if image_size == 512:
  model.load_state_dict(torch.load('512x512_diffusion_uncond_finetune_008100.pt', map_location='cpu'))
else:
  model.load_state_dict(torch.load('256x256_diffusion_uncond.pt', map_location='cpu'))

model.requires_grad_(False).eval().to(device)
for name, param in model.named_parameters():
    if 'qkv' in name or 'norm' in name or 'proj' in name:
        param.requires_grad_()
if model_config['use_fp16']:
    model.convert_to_fp16()

clip_model = clip.load('ViT-B/16', jit=False)[0].eval().requires_grad_(False).to(device)
clip_size = clip_model.visual.input_resolution
normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                 std=[0.26862954, 0.26130258, 0.27577711])
lpips_model = lpips.LPIPS(net='vgg').to(device)


### Actually do the run...

In [ ]:
#@markdown #Input Settings
prompt_input = "17th century Japanese dojo | 4k isometric render | beautiful background skies for| rain drops and lush mountains" #@param {type:"string"}
#@markdown `prompt_input` currently doesn't support weights-
display_rate = 1 #@param {type:"slider", min:1, max:100, step:1} 
#@markdown `display_rate` controls how often 
prompts = "['" + prompt_input + "']"
prompts = [prompt_input]

image_prompts = []
##@markdown Default is blank

n_batches = 1
init_image = None   # This can be an URL or Colab local path and must be in quotes.
skip_timesteps = 0  # This needs to be between approx. 200 and 500 when using an init image.
                    # Higher values make the output look more like the init.
init_scale = 0      # This enhances the effect of the init image, a good value is 1000.
seed = 0

cutn = 32#@param {type:"raw"}
#@markdown `cutn` is the number of randomly cut patches to distort from diffusion. Default 16. 512 model default is 32.
batch_size = 1
if load_512_model:
  cutn_batches = 4  #added, experimental
  cut_pow = 0.5
  #cutn = (cutn * 2)

###########################################
#@markdown ---
#@markdown ##Untested Settings



clip_guidance_scale =  1000#@param {type:"raw"}
#@markdown `clip_guidane scale` controls how much the image should look like the prompt.
tv_scale = 100 #@param {type:"raw"}
#@markdown `tv_scale` Controls the smoothness of the final output.
range_scale = 50 #@param {type:"raw"}
#@markdown `range_scale` controls how far out of range RGB values are allowed to be.
if image_size == 64:
  #when using the 64x64 checkpoint, the cosine noise scheduler is used. 
  #For unclear reasons, this noise scheduler requires different values 
  #for --clip_guidance_scale and --tv_scale. I recommend starting with -cgs 5 -tvs 0.00001 and
  #experimenting from around there. --clip_guidance_scale and --tv_scale will require experimentation.
  tv_scale = (tv_scale / 10000000)
  clip_guidance_scale = (clip_guidance_scale / 200)

###################################################

#@markdown ---
#@markdown ##Real-ESRGAN Upscaling
run_upscaler = True #@param{type:"boolean"}
model_path='/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus.pth' #@param ['/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus_anime_6B.pth','/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus.pth','/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x2plus.pth'] {type:"string"}
#@markdown Highly recommended! `run_upscaler` will run Real-ESRGAN at the end of diffusion.
#upscaling
if model_path == '/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x2plus.pth':
  upscale_value="2"
else:
  upscale_value="4"

#@markdown ---
#@markdown ##Output Settings
image_output_folder = '/content/drive/MyDrive/z_Colab/Simplified-Diffusion_CLIP/images_out' #@param{type:"string"}




###

def do_run_lowres():
    if seed is not None:
        torch.manual_seed(seed)

    make_cutouts = MakeCutouts(clip_size, cutn)
    side_x = side_y = model_config['image_size']

    target_embeds, weights = [], []

    for prompt in prompts:
        txt, weight = parse_prompt(prompt)
        target_embeds.append(clip_model.encode_text(clip.tokenize(txt).to(device)).float())
        weights.append(weight)

    for prompt in image_prompts:
        path, weight = parse_prompt(prompt)
        img = Image.open(fetch(path)).convert('RGB')
        img = TF.resize(img, min(side_x, side_y, *img.size), transforms.InterpolationMode.LANCZOS)
        batch = make_cutouts(TF.to_tensor(img).unsqueeze(0).to(device))
        embed = clip_model.encode_image(normalize(batch)).float()
        target_embeds.append(embed)
        weights.extend([weight / cutn] * cutn)

    target_embeds = torch.cat(target_embeds)
    weights = torch.tensor(weights, device=device)
    if weights.sum().abs() < 1e-3:
        raise RuntimeError('The weights must not sum to 0.')
    weights /= weights.sum().abs()

    init = None
    if init_image is not None:
        init = Image.open(fetch(init_image)).convert('RGB')
        init = init.resize((side_x, side_y), Image.LANCZOS)
        init = TF.to_tensor(init).to(device).unsqueeze(0).mul(2).sub(1)

    cur_t = None

    def cond_fn(x, t, y=None):
        with torch.enable_grad():
            x = x.detach().requires_grad_()
            n = x.shape[0]
            my_t = torch.ones([n], device=device, dtype=torch.long) * cur_t
            out = diffusion.p_mean_variance(model, x, my_t, clip_denoised=False, model_kwargs={'y': y})
            fac = diffusion.sqrt_one_minus_alphas_cumprod[cur_t]
            x_in = out['pred_xstart'] * fac + x * (1 - fac)
            clip_in = normalize(make_cutouts(x_in.add(1).div(2)))
            image_embeds = clip_model.encode_image(clip_in).float()
            dists = spherical_dist_loss(image_embeds.unsqueeze(1), target_embeds.unsqueeze(0))
            dists = dists.view([cutn, n, -1])
            losses = dists.mul(weights).sum(2).mean(0)
            tv_losses = tv_loss(x_in)
            range_losses = range_loss(out['pred_xstart'])
            loss = losses.sum() * clip_guidance_scale + tv_losses.sum() * tv_scale + range_losses.sum() * range_scale
            if init is not None and init_scale:
                init_losses = lpips_model(x_in, init)
                loss = loss + init_losses.sum() * init_scale
            return -torch.autograd.grad(loss, x)[0]

    if model_config['timestep_respacig'].startswith('ddim'):
        sample_fn = diffusion.ddim_sample_loop_progressive
    else:
        sample_fn = diffusion.p_sample_loop_progressive

    for i in range(n_batches):
        cur_t = diffusion.num_timesteps - skip_timesteps - 1

        samples = sample_fn(
            model,
            (batch_size, 3, side_y, side_x),
            clip_denoised=False,
            model_kwargs={},
            cond_fn=cond_fn,
            progress=True,
            skip_timesteps=skip_timesteps,
            init_image=init,
            randomize_class=True,
        )

        for j, sample in enumerate(samples):
            cur_t -= 1
            if j % display_rate == 0 or cur_t == -1:
                print()
                for k, image in enumerate(sample['pred_xstart']):
                    filename = f'progress_{i * batch_size + k:05}.png'
                    TF.to_pil_image(image.add(1).div(2).clamp(0, 1)).save(filename)
                    tqdm.write(f'Batch {i}, step {j}, output {k}:')
                    display.display(display.Image(filename))


def do_run_highres():
    if seed is not None:
        torch.manual_seed(seed)

    make_cutouts = MakeCutouts(clip_size, cutn, cut_pow)
    side_x = side_y = model_config['image_size']

    target_embeds, weights = [], []

    for prompt in prompts:
        txt, weight = parse_prompt(prompt)
        target_embeds.append(clip_model.encode_text(clip.tokenize(txt).to(device)).float())
        weights.append(weight)

    for prompt in image_prompts:
        path, weight = parse_prompt(prompt)
        img = Image.open(fetch(path)).convert('RGB')
        img = TF.resize(img, min(side_x, side_y, *img.size), transforms.InterpolationMode.LANCZOS)
        batch = make_cutouts(TF.to_tensor(img).unsqueeze(0).to(device))
        embed = clip_model.encode_image(normalize(batch)).float()
        target_embeds.append(embed)
        weights.extend([weight / cutn] * cutn)

    target_embeds = torch.cat(target_embeds)
    weights = torch.tensor(weights, device=device)
    if weights.sum().abs() < 1e-3:
        raise RuntimeError('The weights must not sum to 0.')
    weights /= weights.sum().abs()

    init = None
    if init_image is not None:
        init = Image.open(fetch(init_image)).convert('RGB')
        init = init.resize((side_x, side_y), Image.LANCZOS)
        init = TF.to_tensor(init).to(device).unsqueeze(0).mul(2).sub(1)

    cur_t = None

    def cond_fn(x, t, y=None):
        with torch.enable_grad():
            x = x.detach().requires_grad_()
            n = x.shape[0]
            my_t = torch.ones([n], device=device, dtype=torch.long) * cur_t
            out = diffusion.p_mean_variance(model, x, my_t, clip_denoised=False, model_kwargs={'y': y})
            fac = diffusion.sqrt_one_minus_alphas_cumprod[cur_t]
            x_in = out['pred_xstart'] * fac + x * (1 - fac)
            x_in_grad = torch.zeros_like(x_in)
            for i in range(cutn_batches):
                clip_in = normalize(make_cutouts(x_in.add(1).div(2)))
                image_embeds = clip_model.encode_image(clip_in).float()
                dists = spherical_dist_loss(image_embeds.unsqueeze(1), target_embeds.unsqueeze(0))
                dists = dists.view([cutn, n, -1])
                losses = dists.mul(weights).sum(2).mean(0)
                x_in_grad += torch.autograd.grad(losses.sum() * clip_guidance_scale, x_in)[0] / cutn_batches
            tv_losses = tv_loss(x_in)
            range_losses = range_loss(out['pred_xstart'])
            loss = tv_losses.sum() * tv_scale + range_losses.sum() * range_scale
            if init is not None and init_scale:
                init_losses = lpips_model(x_in, init)
                loss = loss + init_losses.sum() * init_scale
            x_in_grad += torch.autograd.grad(loss, x_in)[0]
            grad = -torch.autograd.grad(x_in, x, x_in_grad)[0]
            return grad

    if model_config['timestep_respacing'].startswith('ddim'):
        sample_fn = diffusion.ddim_sample_loop_progressive
    else:
        sample_fn = diffusion.p_sample_loop_progressive

    for i in range(n_batches):
        cur_t = diffusion.num_timesteps - skip_timesteps - 1

        samples = sample_fn(
            model,
            (batch_size, 3, side_y, side_x),
            clip_denoised=False,
            model_kwargs={},
            cond_fn=cond_fn,
            progress=True,
            skip_timesteps=skip_timesteps,
            init_image=init,
            randomize_class=True,
        )

        for j, sample in enumerate(samples):
            #if j % 100 == 0 or cur_t == 0:
            if j % display_rate == 0 or cur_t == 0:

                print()
                for k, image in enumerate(sample['pred_xstart']):
                    filename = f'progress_{i * batch_size + k:05}.png'
                    TF.to_pil_image(image.add(1).div(2).clamp(0, 1)).save(filename)
                    tqdm.write(f'Batch {i}, step {j}, output {k}:')
                    display.display(display.Image(filename))
            cur_t -= 1

##
gc.collect()

unscaled_image = '/content/progress_00000.png'

if load_512_model:
  do_run_highres()
else:
  do_run_lowres
torch.cuda.empty_cache()
if run_upscaler:
  #anKit.install_ESRGAN()
  anKit.image_upscale(model_path,upscale_value,unscaled_image,'/content/')

In [ ]:
#@markdown #Preview upscaled result
from IPython.display import Image
upscaled_image = '/content/progress_00000_out.png'
Image(upscaled_image, width=900, height=900)
dir_make(image_output_folder)
!cp $upscaled_image $image_output_folder
print('Your outputted file is saved as ',image_output_folder)
gc.collect()

In [ ]:
anKit.install_ESRGAN()

This Notebook has been designed so you can "Run All."

Control + F9

If you run out of VRAM (CUDA Error), do a "Reset and Run All."

In [ ]:
%cd $image_output_folder
ls -t | head -n1